In [1]:
pip install sodapy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dask

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install dask-glm

                                              0.0/8.3 MB ? eta -:--:--
     ---                                      0.7/8.3 MB 23.1 MB/s eta 0:00:01
     -------------                            2.7/8.3 MB 34.5 MB/s eta 0:00:01
     ---------------------                    4.5/8.3 MB 36.3 MB/s eta 0:00:01
     --------------------------------         6.8/8.3 MB 39.3 MB/s eta 0:00:01
     ---------------------------------------  8.3/8.3 MB 37.8 MB/s eta 0:00:01
     ---------------------------------------- 8.3/8.3 MB 33.1 MB/s eta 0:00:00
                                              0.0/298.0 kB ? eta -:--:--
     ------------------------------------- 298.0/298.0 kB 18.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dask_ml

                                              0.0/148.7 kB ? eta -:--:--
     ----------                            41.0/148.7 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 148.7/148.7 kB 2.2 MB/s eta 0:00:00
                                              0.0/962.3 kB ? eta -:--:--
     ------------------------------------- 962.3/962.3 kB 30.7 MB/s eta 0:00:00
                                              0.0/2.6 MB ? eta -:--:--
     ------------------------                 1.6/2.6 MB 33.1 MB/s eta 0:00:01
     ---------------------------------------- 2.6/2.6 MB 32.6 MB/s eta 0:00:00
                                              0.0/62.5 kB ? eta -:--:--
     ---------------------------------------- 62.5/62.5 kB 3.3 MB/s eta 0:00:00
                                              0.0/43.3 kB ? eta -:--:--
     ---------------------------------------- 43.3/43.3 kB ? eta 0:00:00
                                              0.0/27.7 MB ? eta -:--:--
     --         

In [1]:
import pandas as pd
from sodapy import Socrata
import pyspark
import json

conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

import dask.dataframe as dd
from dask_glm.datasets import make_regression
from dask_ml.linear_model import LinearRegression

from pyspark.sql.functions import col, when

In [3]:
#test_data_path = '/Users/maludee/repos/Big-Data-Project/data/Parking_Violations_Issued_-_Fiscal_Year_2023.csv'
#open_data_path = '/Users/maludee/repos/Big-Data-Project/data/Open_Parking_and_Camera_Violations.csv'
#precinct_data_path = "/Users/maludee/repos/Big-Data-Project/data/nypp.csv"

# Precinct Map for Tableau

In [3]:
# YEARLY PARKING DATA - FOR TABLEAU
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(test_data_path)

df.repartition(1).write.mode('overwrite').parquet('tmp/parking_2023')

df = spark.read.parquet('/Users/maludee/repos/Big-Data-Project/notebooks/tmp/parking_2023/*.parquet')

23/05/05 23:11:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
df.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation Coun

In [5]:
precinct_keys = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(precinct_data_path)
precinct_keys.show(5)

+--------------------+--------+-------------+---------------+
|            the_geom|Precinct|   Shape_Leng|     Shape_Area|
+--------------------+--------+-------------+---------------+
|MULTIPOLYGON (((-...|       1|80283.5387782|4.72864229826E7|
|MULTIPOLYGON (((-...|       5|18807.1249114|1.80945274385E7|
|MULTIPOLYGON (((-...|       6|24875.9642171|2.20179465474E7|
|MULTIPOLYGON (((-...|       7|17287.5444926| 1.8366669928E7|
|MULTIPOLYGON (((-...|       9|19772.5107407|2.13953862669E7|
+--------------------+--------+-------------+---------------+
only showing top 5 rows



In [6]:
# use sql syntax to parse the nested json
df.createOrReplaceTempView("2023_violations")
query = """
select 
    `Violation Precinct` as violation_precinct,
    count(*) as count_violations
from 
    2023_violations
group by 1
order by 2 desc

"""
precincts = spark.sql(query)
precincts.toPandas().to_csv('violations_by_precinct.csv')

# I think precinct 0 is in error and should be ignored
precincts.show(6)

+------------------+----------------+
|violation_precinct|count_violations|
+------------------+----------------+
|                 0|         5938384|
|                19|          331642|
|                13|          291119|
|                 6|          259154|
|               114|          254978|
|                14|          220291|
+------------------+----------------+
only showing top 6 rows



# ML Section

In [2]:
# OPEN DATASET - FOR MACHINE LEARNING 
df = spark.read.option("inferSchema", "true")\
                  .option("header", "true")\
                  .csv("Open_Parking_and_Camera_Violations.csv", )


In [3]:
df.printSchema()  

root
 |-- Plate: string (nullable = true)
 |-- State: string (nullable = true)
 |-- License Type: string (nullable = true)
 |-- Summons Number: long (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Violation: string (nullable = true)
 |-- Judgment Entry Date: string (nullable = true)
 |-- Fine Amount: double (nullable = true)
 |-- Penalty Amount: double (nullable = true)
 |-- Interest Amount: double (nullable = true)
 |-- Reduction Amount: double (nullable = true)
 |-- Payment Amount: double (nullable = true)
 |-- Amount Due: double (nullable = true)
 |-- Precinct: integer (nullable = true)
 |-- County: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Violation Status: string (nullable = true)
 |-- Summons Image: string (nullable = true)



In [30]:
# select a subset of columns for the model (TODO: add more)
training_df = df.na.drop().select([col("Violation"), col("Fine Amount"), col("Precinct")])
training_df = training_df.filter((col("Precinct") >= "1") & (col("Precinct") <= "124")) #filter out the invalid precints 

#county_df = training_df.groupBy("Precinct").count()
#county_df.toPandas().to_csv("county.csv")


In [34]:
vio_df = training_df.groupBy("Violation").count().sort("count", ascending=False)
vio_df.toPandas().to_csv("violation.csv")

In [53]:
import pandas as pd

# Read CSV file
df = pd.read_csv('violation.csv')

# Replace spaces and "-" with "_"
df.columns = df.columns.str.replace(' ', '_').str.replace('-', '_')

# Define function to convert Violation column to a case statement
def convert_violation_text(text):
    return "case when Violation = '{}' then 1 else 0 end as {},".format(text, text.replace(' ', '_').replace('-', '_'))

# Create a new column with the case statements
df['converted'] = df['Violation'].apply(convert_violation_text)

# Save output as CSV file
df[['converted']].to_csv('sql_vio.csv', index=False)


In [65]:

# make into a temp sql view for convenience 
training_df.createOrReplaceTempView("open_violations")

query = """
select 
    
    case when Precinct >= 1 AND Precinct <= 40 then 1 else 0 end as Manhattan,
    case when Precinct >= 40 AND Precinct <= 60 then 1 else 0 end as Bronx,
    case when Precinct >= 60 AND Precinct <= 100 then 1 else 0 end as Brooklyn,
    case when Precinct >= 100 AND Precinct <= 120 then 1 else 0 end as Queens,
    case when Precinct >= 120 AND Precinct <= 125 then 1 else 0 end as Staten_Island,
    
    case when Violation = 'NO PARKING-STREET CLEANING' then 1 else 0 end as NO_PARKING_STREET_CLEANING,
    case when Violation = 'INSP. STICKER-EXPIRED/MISSING' then 1 else 0 end as INSP_STICKER_EXPIRED_MISSING,
    case when Violation = 'FAIL TO DSPLY MUNI METER RECPT' then 1 else 0 end as FAIL_TO_DSPLY_MUNI_METER_RECPT,
    case when Violation = 'NO STANDING-DAY/TIME LIMITS' then 1 else 0 end as NO_STANDING_DAY_TIME_LIMITS,
    case when Violation = 'FIRE HYDRANT' then 1 else 0 end as FIRE_HYDRANT,
    case when Violation = 'REG. STICKER-EXPIRED/MISSING' then 1 else 0 end as REG_STICKER_EXPIRED_MISSING,
    case when Violation = 'NO PARKING-DAY/TIME LIMITS' then 1 else 0 end as NO_PARKING_DAY_TIME_LIMITS,
    case when Violation = 'DOUBLE PARKING' then 1 else 0 end as DOUBLE_PARKING,
    case when Violation = 'EXPIRED MUNI METER' then 1 else 0 end as EXPIRED_MUNI_METER,
    case when Violation = 'FRONT OR BACK PLATE MISSING' then 1 else 0 end as FRONT_OR_BACK_PLATE_MISSING,
    case when Violation = 'NO STANDING-BUS STOP' then 1 else 0 end as NO_STANDING_BUS_STOP,
    case when Violation = 'NO STANDING-COMM METER ZONE' then 1 else 0 end as NO_STANDING_COMM_METER_ZONE,
    case when Violation = 'FAIL TO DISP. MUNI METER RECPT' then 1 else 0 end as FAIL_TO_DISP_MUNI_METER_RECPT,
    case when Violation = 'NO STANDING-EXC. TRUCK LOADING' then 1 else 0 end as NO_STANDING_EXC_TRUCK_LOADING,
    case when Violation = 'CROSSWALK' then 1 else 0 end as CROSSWALK,
    case when Violation = 'NO STANDING-EXC. AUTH. VEHICLE' then 1 else 0 end as NO_STANDING_EXC_AUTH_VEHICLE,
    case when Violation = 'DOUBLE PARKING-MIDTOWN COMML' then 1 else 0 end as DOUBLE_PARKING_MIDTOWN_COMML,
    case when Violation = 'SIDEWALK' then 1 else 0 end as SIDEWALK,
    case when Violation = 'BIKE LANE' then 1 else 0 end as BIKE_LANE,
    case when Violation = 'NO MATCH-PLATE/STICKER' then 1 else 0 end as NO_MATCH_PLATE_STICKER,
    case when Violation = 'PLTFRM LFTS LWRD POS COMM VEH' then 1 else 0 end as PLTFRM_LFTS_LWRD_POS_COMM_VEH,
    case when Violation = 'SAFETY ZONE' then 1 else 0 end as SAFETY_ZONE,
    case when Violation = 'OBSTRUCTING DRIVEWAY' then 1 else 0 end as OBSTRUCTING_DRIVEWAY,
    case when Violation = 'NO STOPPING-DAY/TIME LIMITS' then 1 else 0 end as NO_STOPPING_DAY_TIME_LIMITS,
    case when Violation = 'COMML PLATES-UNALTERED VEHICLE' then 1 else 0 end as COMML_PLATES_UNALTERED_VEHICLE,
    case when Violation = 'NGHT PKG ON RESID STR-COMM VEH' then 1 else 0 end as NGHT_PKG_ON_RESID_STR_COMM_VEH,
    case when Violation = 'EXPIRED MUNI MTR-COMM MTR ZN' then 1 else 0 end as EXPIRED_MUNI_MTR_COMM_MTR_ZN,
    case when Violation = 'OBSTRUCTING TRAFFIC/INTERSECT' then 1 else 0 end as OBSTRUCTING_TRAFFIC_INTERSECT,
    case when Violation = 'NO STANDING-BUS LANE' then 1 else 0 end as NO_STANDING_BUS_LANE,
    case when Violation = 'INSP STICKER-MUTILATED/CFEIT' then 1 else 0 end as INSP_STICKER_MUTILATED_C_FEIT,
    case when Violation = 'STORAGE-3HR COMMERCIAL' then 1 else 0 end as STORAGE_3HR_COMMERCIAL,
    case when Violation = 'NON-COMPLIANCE W/ POSTED SIGN' then 1 else 0 end as NON_COMPLIANCE_W_POSTED_SIGN,
    case when Violation = 'NO STANDING-TAXI STAND' then 1 else 0 end as NO_STANDING_TAXI_STAND,
    case when Violation = 'NO PARKING-EXC. AUTH. VEHICLE' then 1 else 0 end as NO_PARKING_EXC_AUTH_VEHICLE,
    case when Violation = 'PEDESTRIAN RAMP' then 1 else 0 end as PEDESTRIAN_RAMP,
    case when Violation = 'PARKED BUS-EXC. DESIG. AREA' then 1 else 0 end as PARKED_BUS_EXC_DESIG_AREA,
    case when Violation = 'WRONG WAY' then 1 else 0 end as WRONG_WAY,
    case when Violation = 'SELLING/OFFERING MCHNDSE-METER' then 1 else 0 end as SELLING_OFFERING_MCHNDSE_METER,
    case when Violation = 'EXPIRED METER' then 1 else 0 end as EXPIRED_METER,
    case when Violation = 'NO STANDING-HOTEL LOADING' then 1 else 0 end as NO_STANDING_HOTEL_LOADING,
    case when Violation = 'REG STICKER-MUTILATED/CFEIT' then 1 else 0 end as REG_STICKER_MUTILATED_C_FEIT,
    case when Violation = 'NO STANDING EXCP D/S' then 1 else 0 end as NO_STANDING_EXCP_D_S,
    case when Violation = 'ANGLE PARKING' then 1 else 0 end as ANGLE_PARKING,
    case when Violation = 'IMPROPER REGISTRATION' then 1 else 0 end as IMPROPER_REGISTRATION,
    case when Violation = 'NO PARKING-EXC. HNDICAP PERMIT' then 1 else 0 end as NO_PARKING_EXC_HNDICAP_PERMIT,
    case when Violation = 'TRAFFIC LANE' then 1 else 0 end as TRAFFIC_LANE,
    case when Violation = 'BEYOND MARKED SPACE' then 1 else 0 end as BEYOND_MARKED_SPACE,
    case when Violation = 'NO PARKING-EXC. HOTEL LOADING' then 1 else 0 end as NO_PARKING_EXC_HOTEL_LOADING,
    case when Violation = 'MISSING EQUIPMENT' then 1 else 0 end as MISSING_EQUIPMENT,
    case when Violation = 'FEEDING METER' then 1 else 0 end as FEEDING_METER,
    case when Violation = 'OVERTIME PKG-TIME LIMIT POSTED' then 1 else 0 end as OVERTIME_PKG_TIME_LIMIT_POSTED,
    case when Violation = 'DETACHED TRAILER' then 1 else 0 end as DETACHED_TRAILER,
    case when Violation = 'TUNNEL/ELEVATED/ROADWAY' then 1 else 0 end as TUNNEL_ELEVATED_ROADWAY,
    case when Violation = 'UNAUTHORIZED BUS LAYOVER' then 1 else 0 end as UNAUTHORIZED_BUS_LAYOVER,
    case when Violation = 'NO PARKING-TAXI STAND' then 1 else 0 end as NO_PARKING_TAXI_STAND,
    case when Violation = 'OTHER' then 1 else 0 end as OTHER,
    case when Violation = 'VIN OBSCURED' then 1 else 0 end as VIN_OBSCURED,
    case when Violation = 'IDLING' then 1 else 0 end as IDLING,
    case when Violation = 'NO STD(EXC TRKS/GMTDST NO-TRK)' then 1 else 0 end as NO_STD_EXC_TRKS_GMTDST_NO_TRK,
    case when Violation = 'OVERNIGHT TRACTOR TRAILER PKG' then 1 else 0 end as OVERNIGHT_TRACTOR_TRAILER_PKG,
    case when Violation = 'INTERSECTION' then 1 else 0 end as INTERSECTION,
    case when Violation = 'UNALTERED COMM VEHICLE' then 1 else 0 end as UNALTERED_COMM_VEHICLE,
    case when Violation = 'FAILURE TO DISPLAY BUS PERMIT' then 1 else 0 end as FAILURE_TO_DISPLAY_BUS_PERMIT,
    case when Violation = 'NO STANDING-FOR HIRE VEH STOP' then 1 else 0 end as NO_STANDING_FOR_HIRE_VEH_STOP,
    case when Violation = 'BUS PARKING IN LOWER MANHATTAN' then 1 else 0 end as BUS_PARKING_IN_LOWER_MANHATTAN,
    case when Violation = 'UNAUTHORIZED PASSENGER PICK-UP' then 1 else 0 end as UNAUTHORIZED_PASSENGER_PICK_UP,
    case when Violation = 'NIGHTTIME STD/ PKG IN A PARK' then 1 else 0 end as NIGHTTIME_STD_PKG_IN_A_PARK,
    case when Violation = 'EXCAVATION-VEHICLE OBSTR TRAFF' then 1 else 0 end as EXCAVATION_VEHICLE_OBSTR_TRAFF,
    case when Violation = 'NO STANDING-COMMUTER VAN STOP' then 1 else 0 end as NO_STANDING_COMMUTER_VAN_STOP,
    case when Violation = 'DIVIDED HIGHWAY' then 1 else 0 end as DIVIDED_HIGHWAY,
    case when Violation = 'NO STOP/STANDNG EXCEPT PAS P/U' then 1 else 0 end as NO_STOP_STANDNG_EXCEPT_PAS_P_U,
    case when Violation = 'OT PARKING-MISSING/BROKEN METR' then 1 else 0 end as OT_PARKING_MISSING_BROKEN_METR,
    case when Violation = 'VEHICLE FOR SALE(DEALERS ONLY)' then 1 else 0 end as VEHICLE_FOR_SALE_DEALERS_ONLY,
    case when Violation = 'NO OPERATOR NAM/ADD/PH DISPLAY' then 1 else 0 end as NO_OPERATOR_NAM_ADD_PH_DISPLAY,
    case when Violation = 'ELEVATED/DIVIDED HIGHWAY/TUNNL' then 1 else 0 end as ELEVATED_DIVIDED_HIGHWAY_TUNNL,
    case when Violation = 'ANGLE PARKING-COMM VEHICLE' then 1 else 0 end as ANGLE_PARKING_COMM_VEHICLE,
    case when Violation = 'RAILROAD CROSSING' then 1 else 0 end as RAILROAD_CROSSING,
    case when Violation = 'EXPIRED METER-COMM METER ZONE' then 1 else 0 end as EXPIRED_METER_COMM_METER_ZONE,
    case when Violation = 'ALTERING INTERCITY BUS PERMIT' then 1 else 0 end as ALTERING_INTERCITY_BUS_PERMIT,
    case when Violation = 'WASH/REPAIR VEHCL-REPAIR ONLY' then 1 else 0 end as WASH_REPAIR_VEHCL_REPAIR_ONLY,
    case when Violation = 'PKG IN EXC. OF LIM-COMM MTR ZN' then 1 else 0 end as PKG_IN_EXC_OF_LIM_COMM_MTR_ZN,
    case when Violation = 'VACANT LOT' then 1 else 0 end as VACANT_LOT,
    case when Violation = 'NO STANDING EXCP DP' then 1 else 0 end as NO_STANDING_EXCP_DP,
    case when Violation = 'MIDTOWN PKG OR STD-3HR LIMIT' then 1 else 0 end as MIDTOWN_PKG_OR_STD_3HR_LIMIT,
    case when Violation = 'VEH-SALE/WSHNG/RPRNG/DRIVEWAY' then 1 else 0 end as VEH_SALE_WSHNG_RPRNG_DRIVEWAY,
    case when Violation = 'MARGINAL STREET/WATER FRONT' then 1 else 0 end as MARGINAL_STREET_WATER_FRONT,
    case when Violation = 'OVERTIME STDG D/S' then 1 else 0 end as OVERTIME_STDG_D_S,
    case when Violation = 'BUS LANE VIOLATION' then 1 else 0 end as BUS_LANE_VIOLATION,
    case when Violation = 'UNALTERED COMM VEH-NME/ADDRESS' then 1 else 0 end as UNALTERED_COMM_VEH_NME_ADDRESS,
    case when Violation = 'NO STANDING-OFF-STREET LOT' then 1 else 0 end as NO_STANDING_OFF_STREET_LOT,
    case when Violation = 'NO STANDING-SNOW EMERGENCY' then 1 else 0 end as NO_STANDING_SNOW_EMERGENCY,
    case when Violation = 'REMOVE/REPLACE FLAT TIRE' then 1 else 0 end as REMOVE_REPLACE_FLAT_TIRE,
    case when Violation = 'OVERTIME STANDING DP' then 1 else 0 end as OVERTIME_STANDING_DP,

    `Fine Amount` as fine_amount
    
from 
    open_violations

"""

# query for model training dataset
X_y = spark.sql(query)
X_y.show(1)

+---------+-----+--------+------+-------------+--------------------------+----------------------------+------------------------------+---------------------------+------------+---------------------------+--------------------------+--------------+------------------+---------------------------+--------------------+---------------------------+-----------------------------+-----------------------------+---------+----------------------------+----------------------------+--------+---------+----------------------+-----------------------------+-----------+--------------------+---------------------------+------------------------------+------------------------------+----------------------------+-----------------------------+--------------------+-----------------------------+----------------------+----------------------------+----------------------+---------------------------+---------------+-------------------------+---------+------------------------------+-------------+-------------------------+

In [6]:
# convert to dask - write training set to parquet
X_y.repartition(1).write.mode('overwrite').parquet('tmp/regression_dataset')

Py4JJavaError: An error occurred while calling o46.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.ja

In [28]:
# convert to dask - read from parquet to dask dataframe
X_y = dd.read_parquet('/Users/maludee/repos/Big-Data-Project/notebooks/tmp/regression_dataset/*.parquet')  

X = X_y[['in_state', 'passenger_car']].to_dask_array()
y = X_y['fine_amount'].to_dask_array()

In [29]:
# this is needed to get the arrays to compute? 
X.compute_chunk_sizes()
y.compute_chunk_sizes()

dask.array<values, shape=(96517058,), dtype=float64, chunksize=(96517058,), chunktype=numpy.ndarray>

In [32]:
# train the model
lr = LinearRegression()
lr.fit(X, y)

LinearRegression()

In [35]:
# get predictions
yhat = lr.predict(X)

# why is score returning nan?
lr.score(X, y)

nan

In [39]:
y[:30].compute()

array([ 65.,  50.,  50.,  50.,  50., 115.,  50.,  50.,  50.,  50.,  50.,
        50.,  50.,  50.,  65.,  50.,  45., 115.,  50.,  50.,  65.,  50.,
        50.,  50.,  65.,  65.,  50.,  50.,  50.,  50.])

In [38]:
yhat[:30].compute()


array([-0.3496723 , -0.21716544,  2.07784501,  2.07784501,  2.07784501,
        2.07784501,  2.07784501,  2.07784501,  2.07784501,  2.07784501,
       -0.3496723 ,  2.07784501,  2.07784501,  2.07784501,  2.07784501,
       -0.3496723 ,  2.07784501,  2.07784501,  2.07784501,  2.07784501,
        2.07784501,  2.07784501,  2.07784501, -0.21716544, -0.3496723 ,
       -0.3496723 ,  2.07784501,  2.07784501,  2.07784501, -0.3496723 ])